In [7]:
using Revise, TaylorModels, Plots, TaylorSeries, IntervalArithmetic, IntervalOptimisation
using LazySets:Zonotope,split,Hyperrectangle

In [12]:
function branchandbound(p::TaylorN{T},dom::IntervalBox{N,T},ϵ::Number) where {N,T}
    K = 1
    Rperv = evaluate(p,dom)
    D1,D2 = bisect(dom)
    D = [ D1 , D2 ]
    R = [evaluate(p, D[i]) for i = 1:length(D)]
    Rnext = Interval(minimum(R[i].lo for i = 1:length(R)),
                     maximum(R[i].hi for i = 1:length(R)))
     #condition given in the algo and max no of steps K
    while  (Rperv.hi - Rnext.hi) <= ϵ*(Rnext.hi - Rnext.lo) &&
           (Rperv.lo - Rnext.lo) <= ϵ*(Rnext.hi - Rnext.lo) && (K <= 10)
        #caclulate the max and min of range before further splitting
        Rperv = Interval(minimum(R[i].lo for i = 1:length(R)),
                         maximum(R[i].hi for i = 1:length(R)))
        R_x = [ R[i].hi for i = 1:length(R)]
        R_n = [ R[i].lo for i = 1:length(R)]
        max = maximum(R[i].hi for i = 1:length(R)) # max of range 
        max = findall(x->x == max, R_x)[1]#index of max range giving domain 
        min = minimum(R[i].lo for i = 1:length(R))# min of range
        min = findall(x->x == min, R_n)[1]#index of min range giving domain 
        l_D = length(D[1])#number of dimension
        K = K + 1
        if min == max
            BA = [ ((D[max][i]).hi - (D[max][i]).lo) for i = 1:l_D] #width of the each dimention
            Beta1 = maximum(BA)#max element of BA
            β = findall(x->x==Beta1,BA)[1] #Index of split
            Mat = ones(Int64,length(BA)); Mat[β] = 2 #array of ones and single two
            H = convert(Hyperrectangle,D[max])#convert the domain to be splited into Hyperrectangl
            D1,D2 = split(H,Mat) #now split the Hyperrectangl
            #update the domain array
            D[max] = convert(IntervalBox, D1)
            DD = push!(D[1:max],convert(IntervalBox, D2))
            D = append!(DD, D[(max+1):length(D)])
            #update the range array
            R[max] = evaluate(p,D[max])
            RR = append!(R[1:max], evaluate(p,D[max+1]))
            R = append!(RR, R[(max+1):length(R)])
            #max and min of range after split.
            Rnext = Interval(minimum(R[i].lo for i=1:length(R)),
                             maximum(R[i].hi for i=1:length(R)))
        else
            BA = [ ((D[max][i]).hi - (D[max][i]).lo) for i = 1:l_D] #width of the each dimention
            Beta1 = maximum(BA) #max element of BA
            β = findall(x->x==Beta1,BA)[1]#Index of split
            Mat = ones(Int64,length(BA)); Mat[β] = 2 #array of ones and single two
            H = convert(Hyperrectangle,D[max]) #convert the domain to be splited into Hyperrectangl
            D1,D2 = split(H,Mat) #now split the Hyperrectangl
            #update the domain array
            D[max] = convert(IntervalBox, D1) 
            DD = push!(D[1:max],convert(IntervalBox, D2))
            D = append!(DD, D[(max+1):length(D)])
            #update the range array
            R[max] = evaluate(p,D[max])
            RR = append!(R[1:max], evaluate(p,D[max+1]))
            R = append!(RR, R[(max+1):length(R)])
            #= if index 'max' occurs before the index 'min' then
            update 'min' as the indexing have changed after the update in the domain array.=#         
            if max < min
                min = min + 1
            end
            BA = [ ((D[min][i]).hi - (D[min][i]).lo) for i = 1:l_D]#width of the each dimention
            Beta1 = maximum(BA)  #max element of BA
            β = findall(x->x==Beta1,BA)[1]#Index of split
            Mat = ones(Int64,length(BA)); Mat[β] = 2#array of ones and single two
            H = convert(Hyperrectangle,D[min])#convert the domain to be splited into Hyperrectangl
            D1,D2 = split(H,Mat) #now split the Hyperrectangl
            #update the domain array
            D[min] = convert(IntervalBox, D1)
            DD = push!(D[1:min],convert(IntervalBox, D2))
            D = append!(DD, D[(min+1):length(D)])
            #update the domain array
            R[min] = evaluate(p,D[min])
            RR = append!(R[1:min], evaluate(p,D[min+1]))
            R = append!(RR, R[(min+1):length(R)])
            #max and min of range after split.
            Rnext = Interval(minimum(R[i].lo for i=1:length(R)),
                             maximum(R[i].hi for i=1:length(R)))
        end
    end
    return Interval(minimum(R[i].lo for i=1:length(R)),
                    maximum(R[i].hi for i=1:length(R)))
end

branchandbound (generic function with 1 method)

In [13]:
Dx1=Interval(-4.5,-0.3)
Dx2=Interval(0.4,0.9)
dom=Dx1×Dx2
x1,x2=set_variables(Float64,["x1","x2"],order=5)

himmilbeau=(x1*x1 + x2 - 11)*(x1 * x1 + x2 - 11)
                                        + (x1 + x2*x2 - 7)*(x1 + x2*x2 - 7)
println("Normal eval")
println(evaluate(himmilbeau,dom))

println("BnB")
println(branchandbound(himmilbeau,dom,0.27))


Normal eval
[-344.06, 557.543]
BnB
[-191.551, 296.421]
